In [ ]:
import pandas as pd
import numpy as np
import os
import tkinter
from PIL import ImageTk, Image


from sklearn.preprocessing import LabelEncoder



global_height = 250
global_width = 250

global_divisor = 10


hand_labeled_images = []
hand_labeled_classes = []
current_image_name = None


my_encoder = LabelEncoder()
my_model = None


def image_generator():
    for filename in os.listdir('dogs-cats-images/unknown'):
        yield 'dogs-cats-images/unknown/'+filename
        
my_images = image_generator()


In [ ]:
# https://www.kaggle.com/chetankv/dogs-cats-images

dog_directory = 'dogs-cats-images/dogs'
cat_directory = 'dogs-cats-images/cats'

image_names = []
images_classes = []

for directory in [dog_directory, cat_directory]:

    for filename in os.listdir(directory):
        new_directory = 'dogs-cats-images/unknown'
        image_names.append(str(new_directory)+"/"+str(filename))
        if ('cat' in filename):
            images_classes.append('cat')
        else:
            images_classes.append('dog')
            
true_labels_df = pd.DataFrame({'Image':image_names, 'Class':images_classes})
display(true_labels_df.head())

true_labels_df.to_csv('labels.csv', index=False)

In [ ]:
hand_labels_df = pd.read_csv('hand_labels.csv')
hand_labels_df.head()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
def retrain_model(labeled_images, labeled_classes):
    
    global window
    global labeled_counter
    global global_divisor
    global my_encoder
    global my_model
    
    if (len(labeled_images) == 0):
        return None
    

    if ('cat' not in labeled_classes) or ('dog' not in labeled_classes):
        return my_model
    
    
    if ((len(labeled_images) % global_divisor) > 0):
        return my_model
    else:
        global_divisor *= 2
    
    old_text = labeled_counter.text
    
    labeled_counter.configure(text = "Re-training")
    labeled_counter.text = "Re-training"
    window.update()
    
    images_set = np.zeros((len(labeled_images), 
                           global_height, 
                           global_width, 
                           3))
    
    
    
    
    for i, image_name in enumerate(labeled_images):
        image = Image.open(image_name).resize((global_height, global_width), Image.ANTIALIAS)
        images_set[i][:, :, :] = np.array(image)
        
        
    
    X = images_set.reshape((images_set.shape[0], -1))
    y = my_encoder.fit_transform(labeled_classes)
    
    allowed_depth = int(0.00001*X.shape[1])
    
    
    
    #my_model = DecisionTreeClassifier(max_depth=allowed_depth, min_samples_leaf=2)
    my_model = LogisticRegression()
    my_model.fit(X,y)
    
    labeled_counter.configure(text = old_text)
    labeled_counter.text = old_text
    window.update()
    
    return my_model


#retrain_model(hand_labeled_images, hand_labeled_classes)

In [ ]:
def selection_made_cat():
    selection_made('cat')
    
def selection_made_dog():
    selection_made('dog')

def selection_made(selected_class):
    global label, labeled_counter
    global my_images
    global current_image_name
    global my_model
    global window
    
    #print(current_image, selected_class)
    if (current_image_name is not None):
        hand_labeled_images.append(current_image_name)
        hand_labeled_classes.append(selected_class)
    
    print(len(hand_labeled_images), end='\r')
    
    labeled_counter.configure(text = str(len(hand_labeled_images)))
    labeled_counter.text = str(len(hand_labeled_images))
          
    
    my_model = retrain_model(hand_labeled_images, hand_labeled_classes)
    
    if (my_model is not None):
        print("Using model")
        model_uncertain = False
        while not (model_uncertain):
            current_image_name = next(my_images)
            
            #print("labeling ",current_image_name)
            
            image = Image.open(current_image_name).resize((global_height, global_width), Image.ANTIALIAS)
            image_object = ImageTk.PhotoImage(image)
            label.configure(image=image_object)
            label.image = image_object
            
            new_X = np.array(image).reshape((1, -1))
            predictions = my_model.predict_proba(new_X)[0]
            
            
            
            if (np.max(predictions) <= 0.95):
                print(predictions)
                model_uncertain = True
                image = ImageTk.PhotoImage(Image.open(current_image_name).resize((global_height, global_width), Image.ANTIALIAS))
                label.configure(image=image)
                label.image = image
            else:
                new_y = np.argmax(predictions)
                selected_class = my_encoder.inverse_transform(np.array([new_y]))[0]
                hand_labeled_images.append(current_image_name)
                hand_labeled_classes.append(selected_class)
                print(len(hand_labeled_images), end='\r')
                
            
            labeled_counter.configure(text = str(len(hand_labeled_images)))
            labeled_counter.text = str(len(hand_labeled_images))
            
            my_model = retrain_model(hand_labeled_images, hand_labeled_classes)
            window.update()
                
    else:
        current_image_name = next(my_images)
        image = ImageTk.PhotoImage(Image.open(current_image_name).resize((global_height, global_width), Image.ANTIALIAS))
        label.configure(image=image)
        label.image = image

In [ ]:


# Let's create the Tkinter window
window = tkinter.Tk()
window.title("GUI")

# Finally, to display the image you will make use of the 'Label' method and pass the 'image' variriable as a parameter and use the pack() method to display inside the GUI.
label = tkinter.Label(window, image = None)
label.pack()

labeled_counter = tkinter.Label(window, text="0")
labeled_counter.pack()


tkinter.Button(window, text = "Cat", command=selection_made_cat).pack(side='left')
tkinter.Button(window, text = "Dog", command=selection_made_dog).pack(side='right')


window.mainloop()